In [435]:
import utilities 
import networkx as nx
import json
from itertools import combinations, product
import re

In [436]:
# convert from networkx format to neo4j (not able to convert node attributes at this point):

def nx_to_neo_nodes(graph, return_nodes=True, return_edges=True):
    
    nx_nodes = list(graph.nodes(data=True))
    neo_nodes = ["CREATE " + "(" + re.sub(r'\W+','', nx_nodes[i][0]) + ":" + list(nx_nodes[i][1].keys())[0] + " {" + list(list(nx_nodes[i][1].values())[0].keys())[0] + ":" + '"' + list(list(nx_nodes[i][1].values())[0].values())[0] + '"' + "}" + ")" for i in range(len(nx_nodes))]
    
    nx_edges = list(G.edges.data())
    neo_edges = ["CREATE " + "(" + re.sub(r'\W+','',nx_edges[i][0]) + ")" + "-[:" + list(nx_edges[i][2].keys())[0] + " " + str(list(nx_edges[i][2].values())[0]) + "]" + "->" + "(" + re.sub(r'\W+','',nx_edges[i][1]) + ")" for i in range(len(nx_edges))]
    
    if return_nodes and return_edges:
        neo = neo_nodes + neo_edges
        
    elif return_nodes and not return_edges:
        neo = neo_nodes
        
    elif not return_nodes and return_edges:
        neo = neo_edges    
    
    return neo
    

In [437]:
G = nx.MultiDiGraph()

In [438]:
# open neighborhoods json file

with open('dicts/neighborhood_polys.json','r') as f:
    neighborhoods = json.load(f)


In [439]:
# each neighborhood is a node

for i in range(len(list(neighborhoods.keys()))):
    G.add_node(list(neighborhoods.keys())[i], neighborhood={'name':list(neighborhoods.keys())[i]})
    


In [440]:
# 'is_next_to' edge relationship for neighborhood

hoods = list(G.nodes())
hood_combs = list(combinations(hoods, 2))

for i in range(len(hood_combs)):
  
    if utilities.intersection(neighborhoods, hood_combs[i][0], neighborhoods, hood_combs[i][1]):
        G.add_edge(hood_combs[i][0],hood_combs[i][1], NEXT_TO={}) # relationship is...
        G.add_edge(hood_combs[i][1],hood_combs[i][0], NEXT_TO={}) # bidirectional
            

TopologyException: side location conflict at -87.836580878737252 41.986396112425794
TopologyException: side location conflict at -87.664020791475323 41.947275771403895
TopologyException: side location conflict at -87.648784708051664 41.939987536474959
TopologyException: side location conflict at -87.624576496994393 41.896975362507106
TopologyException: side location conflict at -87.780022286293374 41.997413553897694
TopologyException: side location conflict at -87.780022286293374 41.997413553897694


In [441]:
# save to object

neo_neighborhoods = nx_to_neo_nodes(G, return_nodes=True, return_edges=True)

In [442]:
# reset the graph

G = nx.MultiDiGraph()

In [443]:
# opening census tracts json file

with open('dicts/tract_polys.json','r') as f:
    tracts = json.load(f)


In [444]:
# each tract is a node

for i in range(len(list(tracts.keys()))):
    G.add_node(list(tracts.keys())[i], tract={'name':list(tracts.keys())[i]})
    


In [445]:
# 'is_next_to' edge relationship for tracts

tract_names = list(G.nodes())
tract_combs = list(combinations(tract_names, 2))

for i in range(len(tract_combs)):
  
    if utilities.intersection(tracts, tract_combs[i][0], tracts, tract_combs[i][1]):
        G.add_edge(tract_combs[i][0],tract_combs[i][1], NEXT_TO={}) # relationship is...
        G.add_edge(tract_combs[i][1],tract_combs[i][0], NEXT_TO={}) # bidirectional
            

In [446]:
neo_tracts = nx_to_neo_nodes(G, return_nodes=True, return_edges=True)

In [447]:
# reset the graph

G = nx.MultiDiGraph()

In [448]:
# each neighborhood is a node

for i in range(len(list(neighborhoods.keys()))):
    G.add_node(list(neighborhoods.keys())[i], neighborhood={'name':list(neighborhoods.keys())[i]})
    


In [449]:
# each tract is a node

for i in range(len(list(tracts.keys()))):
    G.add_node(list(tracts.keys())[i], tract={'name':list(tracts.keys())[i]})
    


In [450]:
# 'is_within' and 'contains' edge relationship for tract -> neighborhood

hood_names = list(neighborhoods.keys())
tract_names = list(tracts.keys())

combs = list(product(hood_names, tract_names))

for i in range(len(combs)):
  
    if utilities.intersection(neighborhoods, combs[i][0], tracts, combs[i][1]):
        G.add_edge(combs[i][0],combs[i][1], CONTAINS={}) # relationship is...
        G.add_edge(combs[i][1],combs[i][0], IS_WITHIN={}) # bidirectional
            

In [451]:
neo_contain_relationships = nx_to_neo_nodes(G, return_nodes=False, return_edges=True)

In [452]:
to_neo = neo_neighborhoods + neo_tracts + neo_contain_relationships


In [453]:
# save as txt file

with open('neo.txt', 'w') as neo_text:
    for listitem in to_neo:
        neo_text.write('%s\n' % listitem)